<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter
   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
hh_data = pd.read_csv('data/hh_database.csv', sep=';')
print(hh_data.shape)

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
hh_data.head(4)

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

In [ ]:
hh_data.info()

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
hh_data.isna().sum()

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
hh_data.describe(include='object')

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
def get_education(edu):
    edu_words = edu.split(' ') 
    for i in range(len(edu_words)):
        if edu_words[i] == 'Высшее':  
            return 'высшее'
        if edu_words[i] == 'Неоконченное':
            return 'неоконченное высшее'
        if edu_words[i] == 'Среднее' and edu_words[i+1] == 'специальное':
            return 'среднее специальное'
        if edu_words[i] == 'Среднее' and edu_words[i+1] != 'специальное':
            return 'среднее'        

hh_data['Образование'] = hh_data['Образование и ВУЗ'].apply(get_education)
hh_data['Образование'].value_counts()
hh_data = hh_data.drop(['Образование и ВУЗ'], axis=1)    

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
hh_data['Возраст'] = hh_data['Пол, возраст'].apply(lambda x: int(x.split(' ')[3]))
hh_data['Пол'] = hh_data['Пол, возраст'].apply(lambda x: 'м' if x.split(' ')[0] == 'Мужчина' else 'ж')
hh_data = hh_data.drop(['Пол, возраст'], axis=1)
hh_data['Пол'].value_counts(normalize='True')
hh_data['Возраст'].mean()

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
def work_expirience(arg):
    if arg is np.nan or arg == 'Не указано':
        return None
    arg_splited = arg.split(' ')[:7]
    month_key_words = ['месяц', 'месяца', 'месяцев']
    year_key_words = ['год', 'года', 'лет']
    month = 0
    year = 0
    for i in range(len(arg_splited)):
        if arg_splited[i] in month_key_words:
            month = arg_splited[i-1]
        if arg_splited[i] in year_key_words:
            year = arg_splited[i-1]
       
    return int(year)*12 + int(month)
hh_data['Опыт работы'] = hh_data['Опыт работы'].apply(work_expirience)
hh_data['Опыт работы'].median()


In [ ]:
hh_data.isna().sum()

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
def city_define(arg):
    million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    city = arg.split(' ')[0]
    if city == 'Москва':
        return 'Москва'
    if city == 'Санкт-Петербург':
        return 'Санкт-Петербург'
    if city in million_cities:
        return 'город-миллионник'
    else:
        return 'другое'
hh_data['Город'] = hh_data['Город, переезд, командировки'].apply(city_define)

def readiness_move(arg):
    
    if 'не готов к переезду' in arg:
        return False
    if 'не готова к переезду' in arg:
        return False
    if 'хочу переехать' in arg:
        return True
    else:
        return True
hh_data['Готовность к переезду'] = hh_data['Город, переезд, командировки'].apply(readiness_move)

def readiness_business_trip(arg):
    if 'командировка' in arg:
        if 'не готов к командировкам' in arg:
            return False
        if 'не готова к командировкам' in arg:
            return False
        else:
            return True
    else:
        return False
hh_data['Готовность к командировкам'] = hh_data['Город, переезд, командировки'].apply(readiness_business_trip)


hh_data['Город'].value_counts(normalize=True)

hh_data[(hh_data['Готовность к командировкам']==True) & (hh_data['Готовность к переезду'] == True)].shape[0]*100/hh_data.shape[0]



5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
employment_types = ['полная занятость', 'частичная занятость', 'проектная работа', 'стажировка', 'волонтерство']
work_shedules = ['вахтовый метод', 'гибкий график', 'удаленная работа', 'полный день', 'сменный график']
for employment_type, work_shedule in zip(employment_types, work_shedules):
    hh_data[employment_type] = hh_data['Занятость'].apply(lambda x: employment_type in x)
    hh_data[work_shedule] = hh_data['График'].apply(lambda x: work_shedule in x)
hh_data[(hh_data['проектная работа'] == True) & (hh_data['волонтерство'] == True)].shape[0]
hh_data[(hh_data['вахтовый метод'] == True) & (hh_data['гибкий график'] == True)].shape[0]
hh_data = hh_data.drop(['Занятость', 'График'], axis=1)

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
currency = pd.read_csv('data/ExchangeRates.csv', sep=',')
hh_data['Обновление резюме']= pd.to_datetime(hh_data['Обновление резюме'], dayfirst=True) 
hh_data['Обновление резюме'] = hh_data['Обновление резюме'].dt.date 
currency['date'] = pd.to_datetime(currency['date'], dayfirst=True) #
currency['date'] = currency['date'].dt.date 
hh_data['Заработная плата'] = hh_data['ЗП'].apply(lambda x: x.split(' ')[0]) 
hh_data['Валюта'] = hh_data['ЗП'].apply(lambda x: x.split(' ')[1]) 
hh_data['Валюта'].value_counts() 
hh_data = hh_data.replace({'Валюта': {'бел.руб.':'BYN', 'грн.':'UAH', 'сум':'UZS', 'руб.':'RUR'}})
hh_data_pivot = hh_data.merge(
    currency,
    how='left',
    left_on=['Валюта', 'Обновление резюме'],
    right_on=['currency', 'date']
)
hh_data_pivot['close'] = hh_data_pivot['close'].fillna(1) 
hh_data_pivot['proportion'] = hh_data_pivot['proportion'].fillna(1) 
hh_data_pivot['Заработная плата'] = hh_data_pivot['Заработная плата'].astype(int) 
hh_data_pivot['ЗП в рублях'] = hh_data_pivot['Заработная плата']*hh_data_pivot['close']/hh_data_pivot['proportion']
display(round(hh_data_pivot['ЗП в рублях'].median()/1000))
data = hh_data_pivot.drop(['ЗП', 'proportion', 'close', 'Заработная плата', 'Валюта', 'vol', 'time', 'date', 'per', 'currency', 'Город, переезд, командировки'], axis=1)
data.info()

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
hist_data = data['Возраст']
fig = px.histogram(
    data_frame=hist_data,
x='Возраст',
marginal='box',
title='Распределение признака Возраст'
)
fig.show()

*Медианный возраст соискателей - 31 год. Модальное значение возраста - 30 лет. Минимальный возраст - 14 лет, максимальный - 49 лет. Распределение близко к нормальному, однако сущесвует  аномальное значение - 100 лет. Возраст большинстра соискателей примерно находится в диапазоне от 23 до 35 лет (более половины наблюдений лежит в этом интервале)*

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
hist_data = data['Опыт работы']
fig = px.histogram(
    data_frame=hist_data,
x='Опыт работы',
marginal='box',
title='Распределение признака Опыт работы'
)
fig.show()

*Модальный опыт работы - 80-84 месяца. Минимальный опыт - 1 месяц, максимальный - 299 месяцев. Большинство соискателей имеет опыт работы от 15 до 144 месяцев (в этом интервале лежит большее количество наблюдений). Аномальные значения этого признака - 1188 месяцев (это почти 100 лет, столько невозможно проработать)*

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
hist_data = data['ЗП в рублях']
fig = px.histogram(
    data_frame=hist_data,
x='ЗП в рублях',
marginal='box',
title='Распределение признака заработной платы'
)
fig.show()

*Предельные значения заработной платы - 1 рубль и 180.9 тысяч рублей. Модальное значение - 47.5 - 52.49 тысяч рублей. Заработная плата большинства соискателей примерно находится в интервале от 17.5 тысяч рублей до 102.49 тысяч рублей. Аномальные значения -  7.68 и 24.3 миллиона рублей. Значения  1.75, 2.5 и 3 миллиона рублей, на мой взгляд, реальны.*

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
median_salary = data[data['ЗП в рублях'] < 1000000].groupby('Образование', as_index=False).median()
fig = px.bar(
    data_frame=median_salary,
    x='Образование',
    y='ЗП в рублях',
    title='Зависимость медианной желаемой заработной платы от уровня образования'
)
fig.show()

*Зависимость медианной заработной платы от уровня образования присутствует: чем выше уровень, тем на бОльшую заработную плату соискатель может претендовать. Это и логично, так как хоть уровень образования не увеличивает производительные способности работников, однако является для работодателя сигналом о том, что его способности высоки.
Наибольший уровень желаемой заработной платы наблюдается для соискателей, имеющих высшее образование. Наименьший - для имеющих среднее и среднее специальное образование. 
На мой взгляд, уровень образования, конечно, важен при прогнозировании уровня заработной платы, однако я бы рассматривала его не отдельно, а в комплексе с другими признаками.*

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
fig = px.box(data[data['ЗП в рублях'] < 1000000],
    y='Город',
    x='ЗП в рублях',
    title='Распределение желаемой заработной платы в зависимости от города'
)
fig.show()

*Зависимость заработной платы от признака города, безусловно присутствует. В мегаполисах медианная желаемая заработная плата выше, чем в ругих городах, причем самый высокий уровень в Москве - 85 тысяч рублей. В Санкт-Петербурге  - 60 тысяч рублей. В городах-миллионниках и других городах она одинакова - 40 тысяч рублей. При прогнозировании заработной платы признак города определенно важен. Однако для лаконичности следует рассматривать три категории городов: Москва, Санкт-Петербург и прочие.*

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
fig = px.bar(
data_frame=data.groupby(['Готовность к командировкам', 'Готовность к переезду'], as_index=False).median(),
x='Готовность к переезду',
y='ЗП в рублях',
color='Готовность к командировкам',
text='ЗП в рублях',
orientation='v',
barmode='group',
height=600,
width=1100,
title='Зависимость медианной заработной платы от признаков "Готовность к переезду" и "Готовность к командировкам"'
)
fig.show()

*У соискателей, готовых и к переезду,и к командировкам, уровень желаемой заработной платы самый высокий - почти 66 тысяч рублей. У тех, кто не готов ни к тому, ни к другому, уровень самый низкий - 40 тысяч рублей. Однако у тех, кто готов переехать, но не готов к командировкам желаемая медианная ЗП составляет 50 тысяч рублей. А кто готов к командировкам, но не готов переезжать - 60 тысяч рублей. Возможно, большинство соискателей итак живут в мегаполисе, гду уровень Зп итак высокий, им нет смысла куда-то переезжать, поэтому большей своей ценностью они считают готовность к командировкам. соответственно, ожидаемая заработная плата у них выше, чем у тех, кто живет в провинции и не готов переезжать, хоть и готов ездить в командировки.*

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
pivot_data = data.pivot_table(
    values='ЗП в рублях',
    index='Возраст',
    columns='Образование',
    aggfunc='median',
    fill_value=0
)
heatmap = sns.heatmap(data=pivot_data, cmap='YlGnBu')
heatmap.set_title('Зависимость медианной желаемой заработной платы от возраста и образования', fontsize=16);


*Наблюдаетмя зависимость уровня ожидаемой заработной платы от возраста и уровня образования. В целом самый высокий уровень ожидаемой заработной платы (не считая отдельных выбросов) наблюдается у людей с высшим образованием в возрасте от 38 до 48 лет. Для людей с неоконченным высшим - от 37 до 45 лет. В группах со средним с среднем специальным образованием уровни ожидаемой заработной платы наименьшие.*

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
fig = px.scatter(
    data,
    x='Возраст',
    y=data['Опыт работы'].apply(lambda x: x/12),
    labels={'x':'Возраст', 'y':'Опыт работы (лет)'},
    title='Зависимость опыта работы от возраста'
)
fig.add_shape(type='line',
    x0=0,
    y0=0,
    x1=100,
    y1=100,

)
fig.show()

*Построенная прямая проходит через точки, когда возраст человека равен его опыту. В нашем случае большинство наблюдений лежит ниже этой прямой, нижная граница практически ей параллельна. Это означает что соискатели начинают приобретать опыт примерно в одном и том же возрасте.
Однако 7 точек располагаются выше этой прямой. Это аномальный значения.
Диаграмма показывает, что с увеличением возраста опыт работы растет*

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
fig = plt.figure(figsize=(10, 5))

barplot = sns.barplot(
    data=data,
    x='Пол',
    y='ЗП в рублях',
    color='Lightblue',
    errorbar=None
         
);
barplot.set_title('Соотношение ожидаемой заработной платы женщин и мужчин', fontsize=20)
barplot.set_xlabel('Пол')
barplot.set_ylabel('ЗП в рублях')
barplot.set_title('Зависимость ожидаемой заработной платы от возраста')

fig = px.scatter(
    data,
    x='Возраст',
    y='ЗП в рублях',
    color='Пол',
    title='Зависимость ожидаемой заработной платы от возраста и пола'
)
fig.show()

*Верхняя диаграмма показывает распределение уровня ожидаемой заработной платы в зависимости от возраста в разрезе пола соискателя. Можно сделать вывод, что мужчины раньше начинают работать (с 14 лет), у женщин запросы ЗП начинаютмя с 17 лет. После 60 лет работающих мужчин больше, чем женщин. Максимальный возраст соискателя-женщины 73 года, мужчины - 77 лет.
Нижняя диаграмма показывает зависимость ожидаемой заработрной платы от возраста. У мужчие она выше, чем у женщин. Верхняя диаграмма это тоже подтверждает.*

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
dupl_columns=list(data.columns)
mask = data.duplicated(subset=dupl_columns)
data_duplicates = data[mask]
print(data_duplicates.shape[0])


In [ ]:
data = data.drop_duplicates(subset=dupl_columns)

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
data.isna().sum()

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
drop_data = data.dropna(subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'], how='any', axis=0)
drop_data = drop_data.fillna(value={'Опыт работы': drop_data['Опыт работы'].median()})
drop_data['Опыт работы'].mean()

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
outliers_zarplata = drop_data[(drop_data['ЗП в рублях']>1000000) | (drop_data['ЗП в рублях']<1000)]
print(outliers_zarplata.shape[0])


In [ ]:
drop_data = drop_data.drop(outliers_zarplata.index, axis=0)
drop_data.shape[0]

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
drop_data['Опыт работы в годах'] = drop_data['Опыт работы'].apply(lambda x: x/12)
outliers_exp = drop_data[drop_data['Опыт работы в годах'] > drop_data['Возраст']]
drop_data = drop_data.drop(outliers_exp.index, axis=0)

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
log_age= np.log(drop_data['Возраст']+1)
histplot = sns.histplot(log_age , bins=30)
histplot.set_title('Распределение признака "Возраст" в логарифмическом масштабе')
histplot.axvline(log_age.mean(), color='k', lw=2)
histplot.axvline(log_age.mean()+ 3 * log_age.std(), color='k', ls='--', lw=2)
histplot.axvline(log_age.mean()- 3 * log_age.std(), color='k', ls='--', lw=2);

In [ ]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 4 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

In [ ]:
outliers, cleaned = outliers_z_score(drop_data, 'Возраст', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')



In [ ]:
drop_data = drop_data.drop(outliers.index, axis=0)

На графике видно, что присутствует правосторонняя асимметрия, т.к. среднее значение находится правее моды. Присутствуют наблюдения, которых не входят в интервал - это выбросы